In [ ]:
import os, sys
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.tmaUtils import TMAEventMaker

In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])
instrument = 'LSSTCam'

client = makeEfdClient()
eventMaker = TMAEventMaker()

In [ ]:
dayObs = 20250718
start = Time(f"{dayObsIntToString(dayObs)}T12:00:00")
end = Time(f"{dayObsIntToString(calcNextDay(dayObs))}T12:00:00")
mtmount = getEfdData(
    client,
    "lsst.sal.MTMount.logevent_summaryState",
    columns=['summaryState'],
    begin=start,
    end=end
)
mtmount_fault = mtmount[mtmount['summaryState'] == 3]

mtaos = getEfdData(
    client,
    "lsst.sal.MTAOS.logevent_summaryState",
    columns=['summaryState'],
    begin=start,
    end=end
)
mtaos_fault = mtaos[mtaos['summaryState'] == 3]

mtptg = getEfdData(
    client,
    "lsst.sal.MTPtg.logevent_summaryState",
    columns=['summaryState'],
    begin=start,
    end=end
)
mtptg_fault = mtptg[mtptg['summaryState'] == 3]

mthex = getEfdData(
    client,
    "lsst.sal.MTHexapod.logevent_summaryState",
    columns=['summaryState'],
    begin=start,
    end=end
)
mthex_fault = mthex[mthex['summaryState'] == 3]

mtcamera = getEfdData(
    client,
    "lsst.sal.MTCamera.logevent_summaryState",
    columns=['summaryState'],
    begin=start,
    end=end
)
mtcamera_fault = mtcamera[mtcamera['summaryState'] == 3]

faults = [mtmount_fault, mtaos_fault, mtptg_fault, mthex_fault, mtcamera_fault]
names = ['mtmount', 'mtaos', 'mtptg', 'mthex', 'mtcamera']
print(len(mtmount_fault), len(mtaos_fault), len(mtptg_fault), len(mthex_fault), len(mtcamera_fault))

In [ ]:
exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", 
            where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
print(len(exposureList))
for [id,record] in exposureList:
    if record.observation_type not in ['acq', 'science']:
        continue
    dataId = {'exposure':record.id, 'instrument':'LSSTCam'}
    expRecord = getExpRecordFromDataId(butler, dataId)
    for n, fault in enumerate(faults):
        for fault_time in fault.index:
            exp_begin = expRecord.timespan.begin.utc - TimeDelta(10.0, format='sec')
            exp_end = expRecord.timespan.end.utc + TimeDelta(10.0, format='sec')
            if (fault_time > exp_begin) and (fault_time < exp_end):
                print(f"{names[n]} faulted during {record.id}")